In [1]:
from src.constants import *
from src.model.my_model import MyModel
from tqdm import  tqdm
from monai.networks import one_hot
from src.data.data_loaders import get_loaders
from src.utils.metrics import dice_scores
import numpy as np
import matplotlib.pyplot as plt

<class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
<class 'monai.transforms.utility.array.AsChannelFirst'>: Class `AsChannelFirst` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.


In [2]:
train_loader, test_loader = get_loaders()


Loading dataset:   0%|          | 0/10 [00:00<?, ?it/s]IOStream.flush timed out
IOStream.flush timed out
Loading dataset: 100%|██████████| 10/10 [00:56<00:00,  5.62s/it]


In [3]:

def train_single_epoch(model, optimizer, train_loader, loss_fn):
    losses = []

    for d in train_loader:

        img = d['img'].to(DEVICE)
        mask = d['mask'].to(DEVICE)

        optimizer.zero_grad()

        outputs = model(img)

        outputs = torch.softmax(outputs, dim=1)
        mask = one_hot(mask, num_classes=3)

        loss = loss_fn(outputs, mask)
        loss.backward()
        optimizer.step()

        losses.append(loss.detach().cpu().item())
    print((sum(losses) / len(losses)))
    return model

In [4]:
def vis(image, label, prediction, scores):

    image = image.cpu().numpy()
    label = label.cpu().numpy()
    prediction = prediction.cpu().numpy()

    # Force a 2D example where at least one class is present
    while True:
        # Choose a random slice index
        slice_idx = np.random.randint(0, image.shape[2])

        # Extract the 2D slices
        image_slice = image[0, 0, slice_idx, :, :]
        label_slice = label[0, 0, slice_idx, :, :]
        pred_slice = prediction[0, 0, slice_idx, :, :]

        if len(np.unique(label_slice)) > 1:
            break

    # Plot the slices
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f'Scores of this slice: {scores}')
    ax1.imshow(image_slice)
    ax1.set_title("Image")
    ax2.imshow(label_slice)
    ax2.set_title("Label")
    ax3.imshow(pred_slice)
    ax3.set_title("Prediction")

    # Remove axis ticks
    for ax in [ax1, ax2, ax3]:
        ax.set_xticks([])
        ax.set_yticks([])

    plt.show()


In [5]:
def test_single_epoch(model, test_loader):
    model.eval()
    scores = []

    largest_component = monai.transforms.KeepLargestConnectedComponent()

    visualised = False

    with torch.no_grad():
        for d in test_loader:
            img = d['img'].to(DEVICE)
            mask = d['mask'].to(DEVICE)

            out = monai.inferers.sliding_window_inference(img,
                                                          roi_size=CROP_SIZE,
                                                          sw_batch_size=BATCH_SIZE,
                                                          predictor=model,
                                                          overlap=0.5,
                                                          sw_device=DEVICE,
                                                          device="cpu",
                                                          progress=False,
                                                          )
            out = torch.argmax(out, 1, keepdim=True)
            out = largest_component(out).to(DEVICE)
            s = dice_scores(out, mask)
            print(s)
            scores.append(s)
            vis(img, mask, out, s)

        scores = np.array(scores)
        scores = np.nan_to_num(scores, copy=True, nan=1.0)
        scores = np.sum(scores, axis=0) / scores.shape[0]

    test_score = np.sum(scores) / (len(scores) * 1.0)

In [6]:
torch.cuda.empty_cache()

In [7]:
model = MyModel(in_channels=1,
                 out_channels=3,
                 lower_channels=32,
                 big_channel=16,
                 patch_size=8,
                 embed_dim=256,
                 skip_transformer=False).cuda()

optimizer = torch.optim.Adam(params=model.parameters(), lr=3e-4)
loss_fn = monai.losses.DiceCELoss(lambda_ce=0.4)

In [ ]:
for e in tqdm(range(200)):
    train_single_epoch(model, optimizer, train_loader, loss_fn)

  0%|          | 1/200 [00:46<2:34:52, 46.69s/it]

1.135796308517456


  1%|          | 2/200 [01:31<2:30:38, 45.65s/it]

0.9668397801262992


  2%|▏         | 3/200 [02:16<2:28:57, 45.37s/it]

...

0.6389743413243975


 23%|██▎       | 46/200 [34:32<1:55:39, 45.06s/it]

0.6227420832429613


 24%|██▎       | 47/200 [35:17<1:54:48, 45.02s/it]

0.6568229241030556


In [ ]:
%%time
test_single_epoch(model=model, test_loader=test_loader)

In [ ]:
d =  next(iter(train_loader))

In [ ]:
d['img'].shape